In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import to_date, when
from pyspark.sql.functions import regexp_replace, col
import time

In [ ]:
spark = SparkSession.builder.appName("CNPJ").getOrCreate()

In [ ]:
# Inicializar a sessão Spark

#spark = SparkSession.builder \
#    .appName("CNPJ") \
#    .config("spark.executor.memory", "6g") \
#    .config("spark.executor.cores", "1") \
#    .config("spark.executor.instances", "6") \
#    .config("spark.driver.memory", "4g") \
#    .config("spark.yarn.am.memory", "2g") \
#    .config("spark.network.timeout", "600s") \
#    .config("spark.executor.heartbeatInterval", "100s") \
#    .config("spark.sql.shuffle.partitions", "200") \
#    .getOrCreate()

In [ ]:
# Prefixo do caminho para os arquivos CSV no bucket do Google Cloud Storage
prefix = "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/K3241."
suffix_empresas = ".D40608.EMPRECSV"
suffix_socios = ".D40608.SOCIOCSV"
suffix_estabele = ".D40608.ESTABELE"

# Lista para armazenar os nomes dos arquivos de cada tipo
file_paths_empresas = []
file_paths_socios = []
file_paths_estabele = []

# Gerar os caminhos para os arquivos CSV de EMPRESAS
for i in range(10):
    file_name_empresas = f"{prefix}K03200Y{i}{suffix_empresas}"
    file_paths_empresas.append(file_name_empresas)

# Gerar os caminhos para os arquivos CSV de SOCIOS
for i in range(10):
    file_name_socios = f"{prefix}K03200Y{i}{suffix_socios}"
    file_paths_socios.append(file_name_socios)

# Gerar os caminhos para os arquivos CSV de ESTABELE
for i in range(10):
    file_name_estabele = f"{prefix}K03200Y{i}{suffix_estabele}"
    file_paths_estabele.append(file_name_estabele)

# Imprimir os caminhos dos arquivos
print("EMPRESAS:")
for path in file_paths_empresas:
    print(path)

print("\nSOCIOS:")
for path in file_paths_socios:
    print(path)

print("\nESTABELE:")
for path in file_paths_estabele:
    print(path)


In [ ]:
#IMPORTAR múltiplos arquivos CSV de empresas com um esquema definido, 
# unindo-os em um único DataFrame e criando uma view temporária chamada "EMPRESAS" no Spark.

schema = StructType([
    StructField("CNPJBASICO", StringType(), True),
    StructField("RAZAOSOCIAL", StringType(), True),
    StructField("NATUREZAJURIDICA", StringType(), True),
    StructField("QUALIFICACAODORESPONSAVEL", StringType(), True),
    StructField("CAPITALSOCIAL", StringType(), True),
    StructField("PORTE", StringType(), True),
    StructField("ENTEFEDERATIVO", StringType(), True)
])

# Ler e unir todos os arquivos CSV em um DataFrame
df_list = [spark.read.csv(file_path, header=False, schema=schema, sep=';') for file_path in file_paths_empresas]
df = df_list[0]
for df_part in df_list[1:]:
    df = df.union(df_part)
    
df = df.withColumn("CAPITALSOCIAL", regexp_replace(col("CAPITALSOCIAL"), ",", ".").cast(DoubleType()))
# Traduzir valores do porte da empresa
df = df.withColumn("PORTE", when(col("PORTE") == '00', 'NÃO INFORMADO')
                             .when(col("PORTE") == '01', 'MICRO EMPRESA')
                             .when(col("PORTE") == '03', 'EMPRESA DE PEQUENO PORTE')
                             .when(col("PORTE") == '05', 'DEMAIS')
                             .otherwise('NÃO INFORMADO'))

# Criar uma view temporária
df.createOrReplaceTempView("EMPRESAS")

In [ ]:
#IMPORTAR múltiplos arquivos CSV de ESTABELECIMENTOS com um esquema definido, 
# unindo-os em um único DataFrame e criando uma view temporária chamada "ESTABELECIMENTOS" no Spark.

schema_estabele = StructType([
    StructField("CNPJBASICO", StringType(), True),
    StructField("CNPJORDEM", StringType(), True),
    StructField("CNPJDV", StringType(), True),
    StructField("IDENTIFICADORMATRIZFILIAL", StringType(), True),
    StructField("NOMEFANTASIA", StringType(), True),
    StructField("SITUACAOCADASTRAL", StringType(), True),
    StructField("DATASITUACAOCADASTRAL", StringType(), True),
    StructField("MOTIVOSITUACAOCADASTRAL", StringType(), True),
    StructField("NOMECIDADEEXTERIOR", StringType(), True),
    StructField("PAIS", StringType(), True),
    StructField("DATAINICIOATIVIDADE", StringType(), True),
    StructField("CNAEFISCALPRINCIPAL", StringType(), True),
    StructField("CNAEFISCALSECUNDARIA", StringType(), True),
    StructField("TIPOLOGRADOURO", StringType(), True),
    StructField("LOGRADOURO", StringType(), True),
    StructField("NUMERO", StringType(), True),
    StructField("COMPLEMENTO", StringType(), True),
    StructField("BAIRRO", StringType(), True),
    StructField("CEP", StringType(), True),
    StructField("UF", StringType(), True),
    StructField("MUNICIPIO", StringType(), True),
    StructField("DDD1", StringType(), True),
    StructField("TELEFONE1", StringType(), True),
    StructField("DDD2", StringType(), True),
    StructField("TELEFONE2", StringType(), True),
    StructField("DDDFAX", StringType(), True),
    StructField("FAX", StringType(), True),
    StructField("EMAIL", StringType(), True),
    StructField("SITUACAOESPECIAL", StringType(), True),
    StructField("DATASITUACAOESPECIAL", StringType(), True),
])

# Ler e unir todos os arquivos CSV de ESTABELECIMENTOS em um DataFrame
df_list_estabele = [spark.read.csv(file_path, header=False, schema=schema_estabele, sep=';') for file_path in file_paths_estabele]
df_estabele = df_list_estabele[0]
for df_part in df_list_estabele[1:]:
    df_estabele = df_estabele.union(df_part)
    
# Transformar campos de data
df_estabele = df_estabele.withColumn("DATASITUACAOCADASTRAL", to_date(df_estabele["DATASITUACAOCADASTRAL"], "yyyyMMdd"))
df_estabele = df_estabele.withColumn("DATAINICIOATIVIDADE", to_date(df_estabele["DATAINICIOATIVIDADE"], "yyyyMMdd"))
df_estabele = df_estabele.withColumn("DATASITUACAOESPECIAL", to_date(df_estabele["DATASITUACAOESPECIAL"], "yyyyMMdd"))
# Traduzir valores do identificador matriz/filial
df_estabele = df_estabele.withColumn("IDENTIFICADORMATRIZFILIAL", 
                                     when(col("IDENTIFICADORMATRIZFILIAL") == '1', 'MATRIZ')
                                     .when(col("IDENTIFICADORMATRIZFILIAL") == '2', 'FILIAL')
                                     .otherwise('NÃO INFORMADO'))

# Traduzir valores da situação cadastral
df_estabele = df_estabele.withColumn("SITUACAOCADASTRAL", 
                                     when(col("SITUACAOCADASTRAL") == '01', 'NULA')
                                     .when(col("SITUACAOCADASTRAL") == '02', 'ATIVA')
                                     .when(col("SITUACAOCADASTRAL") == '03', 'SUSPENSA')
                                     .when(col("SITUACAOCADASTRAL") == '04', 'INAPTA')
                                     .when(col("SITUACAOCADASTRAL") == '08', 'BAIXADA')
                                     .otherwise('NÃO INFORMADO'))



# Eliminar o caractere '\' em NUMERO e COMPLEMENTO
df_estabele = df_estabele.withColumn("NUMERO", regexp_replace(col("NUMERO"), r'\\', ''))
df_estabele = df_estabele.withColumn("COMPLEMENTO", regexp_replace(col("COMPLEMENTO"), r'\\', ''))
df_estabele = df_estabele.withColumn("LOGRADOURO", regexp_replace(col("LOGRADOURO"), r'\\', ''))
df_estabele = df_estabele.withColumn("LOGRADOURO", regexp_replace(col("LOGRADOURO"), r'\\x00', ''))


# Criar uma view temporária
df_estabele.createOrReplaceTempView("ESTABELECIMENTOS")




In [ ]:
#IMPORTAR múltiplos arquivos CSV de socioS com um esquema definido, 
# unindo-os em um único DataFrame e criando uma view temporária chamada "SÓCIOS" no Spark.

schema_socios = StructType([
    StructField("CNPJBASICO", StringType(), True),
    StructField("IDENTIFICADORSOCIO", StringType(), True),
    StructField("NOMESOCIORAZAOSOCIAL", StringType(), True),
    StructField("CNPJCPFSOCIO", StringType(), True),
    StructField("QUALIFICACAOSOCIO", StringType(), True),
    StructField("DATAENTRADASOCIEDADE", StringType(), True),
    StructField("PAIS", StringType(), True),
    StructField("CPFREPRESENTANTE", StringType(), True),
    StructField("NOMEREPRESENTANTE", StringType(), True),
    StructField("QUALIFICACAOREPRESENTANTE", StringType(), True),
    StructField("FAIXAETARIA", StringType(), True)
])

# Ler e unir todos os arquivos CSV de SOCIOS em um DataFrame
df_list_socios = [spark.read.csv(file_path, header=False, schema=schema_socios, sep=';') for file_path in file_paths_socios]
df_socios = df_list_socios[0]
for df_part in df_list_socios[1:]:
    df_socios = df_socios.union(df_part)
    
# Transformar campos de data
df_socios = df_socios.withColumn("DATAENTRADASOCIEDADE", to_date(df_socios["DATAENTRADASOCIEDADE"], "yyyyMMdd"))
# Traduzir valores do identificador de sócio
df_socios = df_socios.withColumn("IDENTIFICADORSOCIO", 
                                 when(col("IDENTIFICADORSOCIO") == '1', 'PESSOA JURÍDICA')
                                 .when(col("IDENTIFICADORSOCIO") == '2', 'PESSOA FÍSICA')
                                 .when(col("IDENTIFICADORSOCIO") == '3', 'ESTRANGEIRO')
                                 .otherwise('NÃO INFORMADO'))


# Traduzir valores do identificador de sócio
# Atualizar a coluna "PAIS" para 'BRASIL' onde estiver nula ou vazia
df_socios = df_socios.withColumn(
    "PAIS", 
    when((col("PAIS") == '') | col("PAIS").isNull() | (col("PAIS") =='NULL'), 'BRASIL')
    .otherwise(col("PAIS"))
)


df_socios = df_socios.withColumn("FAIXAETARIA", 
                                 when(col("FAIXAETARIA") == '0', '0-12')
                                 .when(col("FAIXAETARIA") == '1', '13-20')
                                 .when(col("FAIXAETARIA") == '2', '21-30')
                                 .when(col("FAIXAETARIA") == '3', '31-40')
                                 .when(col("FAIXAETARIA") == '4', '41-50')
                                 .when(col("FAIXAETARIA") == '5', '51-60')
                                 .when(col("FAIXAETARIA") == '6', '61-70')
                                 .when(col("FAIXAETARIA") == '7', '71-80')
                                 .when(col("FAIXAETARIA") == '8', '81+')
                                 .otherwise('NÃO INFORMADO'))


# Criar uma view temporária
df_socios.createOrReplaceTempView("SOCIOS")

# Verificar os dados e esquema do DataFrame
#df_socios.show()
#df_socios.printSchema()

In [ ]:
#Este código lê o arquivo CSV em um DataFrame Spark 
# e cria uma view temporária chamada "SIMPLES".

file_path_simples = "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/F.K03200$W.SIMPLES.CSV.D40608"
# Esquema para o arquivo SIMPLES
schema_simples = StructType([
    StructField("CNPJBASICO", StringType(), True),
    StructField("OPCAOPELOSIMPLES", StringType(), True),
    StructField("DATAOPCAOPELOSIMPLES", StringType(), True),
    StructField("DATAEXCLUSAODOSIMPLES", StringType(), True),
    StructField("OPCAOPELOMEI", StringType(), True),
    StructField("DATAOPCAOPELOMEI", StringType(), True),
    StructField("DATAEXCLUSAODOMEI", StringType(), True)
])

# Ler o arquivo CSV de SIMPLES em um DataFrame
df_simples = spark.read.csv(file_path_simples, header=False, schema=schema_simples, sep=';')

# Transformar campos de data
df_simples = df_simples.withColumn("DATAOPCAOPELOSIMPLES", to_date(df_simples["DATAOPCAOPELOSIMPLES"], "yyyyMMdd"))
df_simples = df_simples.withColumn("DATAEXCLUSAODOSIMPLES", to_date(df_simples["DATAEXCLUSAODOSIMPLES"], "yyyyMMdd"))
df_simples = df_simples.withColumn("DATAOPCAOPELOMEI", to_date(df_simples["DATAOPCAOPELOMEI"], "yyyyMMdd"))
df_simples = df_simples.withColumn("DATAEXCLUSAODOMEI", to_date(df_simples["DATAEXCLUSAODOMEI"], "yyyyMMdd"))
# Traduzir valores 'S' e 'N' para 'SIM' e 'NÃO'
df_simples = df_simples.withColumn("OPCAOPELOSIMPLES", when(df_simples["OPCAOPELOSIMPLES"] == 'S', 'SIM').otherwise('NÃO'))
df_simples = df_simples.withColumn("OPCAOPELOMEI", when(df_simples["OPCAOPELOMEI"] == 'S', 'SIM').otherwise('NÃO'))



# Criar uma view temporária
df_simples.createOrReplaceTempView("SIMPLES")

# Verificar os dados e esquema do DataFrame
#df_simples.show()
#df_simples.printSchema()

In [ ]:
#Este código carrega várias tabelas de domínio (CNAE, MOTI, MUNIC, NATJU, PAIS, QUALS)
#a partir de arquivos CSV armazenados no Google Cloud Storage, 
#utilizando esquemas específicos e a codificação ISO-8859-1, e cria views temporárias no Spark para cada uma dessas tabelas.

# CARREGANDO AS TABELAS DE DOMÍNIO
file_paths_codigos = {
    "CNAE": "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/F.K03200$Z.D40608.CNAECSV",
    "MOTI": "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/F.K03200$Z.D40608.MOTICSV",
    "MUNIC": "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/F.K03200$Z.D40608.MUNICCSV",
    "NATJU": "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/F.K03200$Z.D40608.NATJUCSV",
    "PAIS": "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/F.K03200$Z.D40608.PAISCSV",
    "QUALS": "gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/F.K03200$Z.D40608.QUALSCSV"
}

# Esquemas para os arquivos CSV das tabelas de códigos
schemas_codigos = {
    "CNAE": StructType([StructField("CNAE_CODIGO", StringType(), True), StructField("CNAE_DESCRICAO", StringType(), True)]),
    "MOTI": StructType([StructField("MOTIVO_CODIGO", StringType(), True), StructField("MOTIVO_DESCRICAO", StringType(), True)]),
    "MUNIC": StructType([StructField("MUNICIPIO_CODIGO", StringType(), True), StructField("MUNICIPIO_DESCRICAO", StringType(), True)]),
    "NATJU": StructType([StructField("NATJU_CODIGO", StringType(), True), StructField("NATJU_DESCRICAO", StringType(), True)]),
    "PAIS": StructType([StructField("PAIS_CODIGO", StringType(), True), StructField("PAIS_DESCRICAO", StringType(), True)]),
    "QUALS": StructType([StructField("QUALS_CODIGO", StringType(), True), StructField("QUALS_DESCRICAO", StringType(), True)])
}  
    
    # Função para carregar os DataFrames de códigos com especificação de codificação
def load_codigos(file_path, schema, encoding="UTF-8"):
    return spark.read.csv(file_path, header=False, schema=schema, sep=';', encoding=encoding)

# Carregar os DataFrames de códigos com especificação de codificação
df_codigos = {key: load_codigos(file_paths_codigos[key], schemas_codigos[key], encoding="ISO-8859-1") for key in file_paths_codigos}

# Criar views temporárias para as tabelas de códigos
for key, df in df_codigos.items():
    df.createOrReplaceTempView(key)
    
#Como algumas consultas podem ter a descrição do CNAE em vez do numero, vamos criar essa tabela e usar para indexar os LLM
    
df_cnaes = spark.sql("""
    SELECT 
        c.CNAE_CODIGO,
        c.CNAE_DESCRICAO
    FROM CNAE c
""")

df_cnaes.cache()

# Trigger an action to force caching
df_cnaes.count()

In [ ]:
#SUBSTITUINDO NA TABELA EMPRESAS OS CODIGOS PELAS DESCRIÇÕES DAS TABELAS DE DOMINIO
# Substituir códigos pelas descrições nas tabelas EMPRESAS
df_empresas_desc = spark.sql("""
    SELECT 
        e.CNPJBASICO,
        e.RAZAOSOCIAL,
        nj.NATJU_DESCRICAO as NATUREZAJURIDICA,
        q.QUALS_DESCRICAO as QUALIFICACAODORESPONSAVEL,
        e.CAPITALSOCIAL,
        e.PORTE,
        e.ENTEFEDERATIVO
    FROM EMPRESAS e
    LEFT JOIN NATJU nj ON e.NATUREZAJURIDICA = nj.NATJU_CODIGO
    LEFT JOIN QUALS q ON e.QUALIFICACAODORESPONSAVEL = q.QUALS_CODIGO
""")


df_empresas_desc.cache()

# Trigger an action to force caching
df_empresas_desc.count()

# Create a view from the cached DataFrame
df_empresas_desc.createOrReplaceTempView("EMPRESAS")



In [ ]:
# Substituir códigos pelas descrições nas tabelas ESTABELECIMENTOS
df_estabele_desc = spark.sql("""
    SELECT 
        e.CNPJBASICO,
        e.CNPJORDEM,
        e.CNPJDV,
        e.IDENTIFICADORMATRIZFILIAL,
        e.NOMEFANTASIA,
        e.SITUACAOCADASTRAL,
        e.DATASITUACAOCADASTRAL,
        m.MOTIVO_DESCRICAO as MOTIVOSITUACAOCADASTRAL,
        e.NOMECIDADEEXTERIOR,
        p.PAIS_DESCRICAO as PAIS,
        e.DATAINICIOATIVIDADE,
        c.CNAE_DESCRICAO as CNAEFISCALPRINCIPAL,
        e.CNAEFISCALSECUNDARIA,
        e.TIPOLOGRADOURO,
        e.LOGRADOURO,
        e.NUMERO,
        e.COMPLEMENTO,
        e.BAIRRO,
        e.CEP,
        e.UF,
        mu.MUNICIPIO_DESCRICAO as MUNICIPIO,
        e.DDD1,
        e.TELEFONE1,
        e.DDD2,
        e.TELEFONE2,
        e.DDDFAX,
        e.FAX,
        e.EMAIL,
        e.SITUACAOESPECIAL,
        e.DATASITUACAOESPECIAL
    FROM ESTABELECIMENTOS e
    LEFT JOIN MOTI m ON e.MOTIVOSITUACAOCADASTRAL = m.MOTIVO_CODIGO
    LEFT JOIN PAIS p ON e.PAIS = p.PAIS_CODIGO
    LEFT JOIN CNAE c ON e.CNAEFISCALPRINCIPAL = c.CNAE_CODIGO
    LEFT JOIN MUNIC mu ON e.MUNICIPIO = mu.MUNICIPIO_CODIGO
    wHERE CNPJBASICO != 13269396
""")

df_estabele_desc.cache()

# Trigger an action to force caching
df_estabele_desc.count()

df_estabele_desc.createOrReplaceTempView("ESTABELECIMENTOS")
#df_estabele_desc.printSchema()


In [ ]:
# Substituir códigos pelas descrições nas tabelas SOCIOS

df_socios_desc = spark.sql("""
    SELECT 
        s.CNPJBASICO,
        s.IDENTIFICADORSOCIO,
        s.NOMESOCIORAZAOSOCIAL,
        s.CNPJCPFSOCIO,
        q.QUALS_DESCRICAO as QUALIFICACAOSOCIO,
        s.DATAENTRADASOCIEDADE,
        p.PAIS_DESCRICAO as PAIS,
        s.CPFREPRESENTANTE,
        s.NOMEREPRESENTANTE,
        qr.QUALS_DESCRICAO as QUALIFICACAOREPRESENTANTE,
        s.FAIXAETARIA
    FROM SOCIOS s
    LEFT JOIN QUALS q ON s.QUALIFICACAOSOCIO = q.QUALS_CODIGO
    LEFT JOIN PAIS p ON s.PAIS = p.PAIS_CODIGO
    LEFT JOIN QUALS qr ON s.QUALIFICACAOREPRESENTANTE = qr.QUALS_CODIGO
""")

# Atualizar a coluna "PAIS" para 'BRASIL' onde estiver nula ou vazia
df_socios_desc = df_socios_desc.withColumn(
    "PAIS", 
    when((col("PAIS") == '') | col("PAIS").isNull(), 'BRASIL')
    .otherwise(col("PAIS"))
)

# Criar uma view temporária
df_socios.createOrReplaceTempView("SOCIOS")


df_socios_desc.cache()

# Trigger an action to force caching
df_socios_desc.count()

df_socios_desc.createOrReplaceTempView("SOCIOS")
#df_socios_desc.printSchema()


In [ ]:
import pandas as pd

# Função para obter detalhes da view
def get_view_details(view_name):
    df = spark.sql(f"SELECT * FROM {view_name}")
    num_rows = df.count()
    num_columns = len(df.columns)
    schema = df._jdf.schema().treeString()  # Obter o schema formatado com quebras de linha
    return (view_name, num_rows, num_columns, schema)

# Obter detalhes de cada view
views = ["EMPRESAS", "ESTABELECIMENTOS", "SOCIOS", "SIMPLES","CNAE"]
details = [get_view_details(view) for view in views]

# Criar DataFrame com os detalhes
df_details = pd.DataFrame(details, columns=["View", "Number of Rows", "Number of Columns", "Schema"])

# Exibir a tabela com schemas formatados
for index, row in df_details.iterrows():
    print(f"View: {row['View']}")
    print(f"Number of Rows: {row['Number of Rows']}")
    print(f"Number of Columns: {row['Number of Columns']}")
    print(f"Schema:\n{row['Schema']}\n")

# Calcular e imprimir a quantidade total de linhas
total_rows = df_details["Number of Rows"].sum()
print(f"\nTotal Number of Rows: {total_rows}")


In [ ]:
# Salvar a view EMPRESAS como CSV
df_empresas = spark.sql("SELECT * FROM EMPRESAS")
df_empresas.coalesce(1).write.csv("gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/DadosProcessados/empresas.csv", header=True, sep=";", mode="overwrite")


In [ ]:
#Salvando os arquivos como CSV para poder importar localmente
# Salvar a view ESTABELECIMENTOS como CSV
df_estabelecimentos = spark.sql("SELECT * FROM ESTABELECIMENTOS WHERE CNPJBASICO != 13269396")
df_estabelecimentos.coalesce(1).write.csv("gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/DadosProcessados/estabelecimentos.csv", header=True, sep=";", mode="overwrite")

In [ ]:
# Salvar a view SOCIOS como CSV
df_socios = spark.sql("SELECT * FROM SOCIOS")
df_socios.coalesce(1).write.csv("gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/DadosProcessados/socios.csv", header=True, sep=";", mode="overwrite")

# Salvar a view SIMPLES como CSV
df_simples = spark.sql("SELECT * FROM SIMPLES")
df_simples.coalesce(1).write.csv("gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/DadosProcessados/simples.csv", header=True, sep=";", mode="overwrite")

# Salvar a view CNAE como CSV
df_cnaes.coalesce(1).write.csv("gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/DadosProcessados/cnaes.csv", header=True, sep=";", mode="overwrite")

In [ ]:
# Função para parar a sessão do Spark
def stop_spark_session(spark_session_name):
    # Verificar se a sessão existe e pará-la
    if spark_session_name in locals():
        locals()[spark_session_name].stop()
        print(f"Sessão {spark_session_name} parada.")

stop_spark_session('CNPJ')

In [ ]:
stop_spark_session('CNPJ')

In [ ]:
# Salvar a view SIMPLES como CSV
df_simples = spark.sql("SELECT * FROM SIMPLES")
df_simples.coalesce(1).write.csv("gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/DadosProcessados/simples.csv", header=True, sep=";", mode="overwrite")

# Salvar a view CNAE como CSV
df_cnaes.coalesce(1).write.csv("gs://dataproc-staging-us-central1-574457499229-8awbbdyx/notebooks/jupyter/DADOS/DadosProcessados/cnaes.csv", header=True, sep=";", mode="overwrite")